# Encoder

> Architectures and functions for creating encoders that create the embeddings

In [1]:
#export
import pandas as pd
import numpy as np
from fastcore.all import *
from tsai.callback.MVP import *
from tsai.imports import *
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.models.explainability import get_acts_and_grads
from tsai.models.layers import *
from tsai.data.validation import combine_split_data

Initialize loss function


/home/macu/env/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/macu/env/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#hide
from tsai.all import *

Initialize loss function
Initialize loss function


### Architectures

In [3]:
#export 
class DCAE_torch(Module):
    def __init__(self, c_in, seq_len, delta, nfs=[64, 32, 12], kss=[10, 5, 5],
                 pool_szs=[2,2,3], output_fsz=10):
        """
        Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions,
        sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be
        contained in the Dense layer of the network. The the number of features
        maps (filters), the filter size and the pool size can also be adjusted."
        """
        assert all_equal([len(x) for x in [nfs, kss, pool_szs]], np.repeat(len(nfs), 3)), \
            'nfs, kss, and pool_szs must have the same length'
        assert np.prod(pool_szs) == nfs[-1], \
            'The number of filters in the last conv layer must be equal to the product of pool sizes'
        assert seq_len % np.prod(pool_szs) == 0, \
            'The product of pool sizes must be a divisor of the window size'
        layers = []
        for i in range_of(kss):
            layers += [Conv1d(ni=nfs[i-1] if i>0 else c_in, nf=nfs[i], ks=kss[i]),
                       nn.MaxPool1d(kernel_size=pool_szs[i])]
        self.downsample = nn.Sequential(*layers)
        self.bottleneck = nn.Sequential(OrderedDict([
            ('flatten', nn.Flatten()),
            ('latent_in', nn.Linear(seq_len, delta)),
            ('latent_out', nn.Linear(delta, seq_len)),
            ('reshape', Reshape(nfs[-1], seq_len // np.prod(pool_szs)))
        ]))
        layers = []
        for i in reversed(range_of(kss)):
            layers += [Conv1d(ni=nfs[i+1] if i != (len(nfs)-1) else nfs[-1],
                              nf=nfs[i], ks=kss[i]),
                       nn.Upsample(scale_factor=pool_szs[i])]
        layers += [Conv1d(ni=nfs[0], nf=c_in, kernel_size=output_fsz)]
        self.upsample = nn.Sequential(*layers)


    def forward(self, x):
        print(x)
        x = self.downsample(x)
        x = self.bottleneck(x)
        x = self.upsample(x)
        return x

In [4]:
import nbs.orelm.orelm_torch as orelm

In [5]:
foo = torch.rand(3, 1, 48)
m = DCAE_torch(c_in=foo.shape[1], seq_len=foo.shape[2], delta=12)
m(foo).shape

tensor([[[0.2187, 0.2202, 0.0252, 0.6997, 0.7276, 0.8556, 0.3818, 0.1662,
          0.9871, 0.8370, 0.2446, 0.5828, 0.3632, 0.3182, 0.0349, 0.1342,
          0.7396, 0.5576, 0.2320, 0.8649, 0.6817, 0.3262, 0.1299, 0.0337,
          0.6044, 0.1846, 0.4219, 0.0439, 0.8326, 0.0328, 0.1613, 0.9337,
          0.9161, 0.8289, 0.6837, 0.2572, 0.6762, 0.2782, 0.6238, 0.2678,
          0.9351, 0.1212, 0.5772, 0.3518, 0.0787, 0.0834, 0.2634, 0.1563]],

        [[0.3331, 0.5990, 0.1588, 0.5235, 0.6795, 0.2225, 0.2564, 0.2995,
          0.2319, 0.6948, 0.6322, 0.9190, 0.0130, 0.0851, 0.4698, 0.5151,
          0.7629, 0.1023, 0.4121, 0.1894, 0.4015, 0.3342, 0.5738, 0.0096,
          0.4897, 0.6656, 0.2578, 0.1501, 0.0547, 0.4989, 0.3745, 0.3645,
          0.1880, 0.9236, 0.3587, 0.6233, 0.2172, 0.4557, 0.8082, 0.6647,
          0.1445, 0.1452, 0.8362, 0.0592, 0.5537, 0.6958, 0.0469, 0.1944]],

        [[0.4702, 0.2869, 0.3401, 0.9439, 0.1309, 0.9688, 0.1241, 0.3421,
          0.4349, 0.8342, 0.0289

torch.Size([3, 1, 48])

### Dictionary to get the default backbone modules to get the embeddings from

In [6]:
#export
ENCODER_EMBS_MODULE_NAME = {
    InceptionTimePlus: 'backbone', # for mvp based models
    DCAE_torch: 'bottleneck.latent_in',
    orelm.ORELM_torch: 'outputs'
}

### Getting the embeddings (activations) from the encoder

In [7]:
#export
def get_enc_embs(X, enc_learn, module=None, cpu=False, average_seq_dim=True, to_numpy=True):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
    """
    if cpu:
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        enc_learn.dls.cuda()
        enc_learn.cuda()
    if enc_learn.dls.bs == 0: enc_learn.dls.bs = 64
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    aux_dl.bs = enc_learn.dls.bs if enc_learn.dls.bs>0 else 64
    module = nested_attr(enc_learn.model,
                         ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) \
                if module is None else module
    embs = [get_acts_and_grads(model=enc_learn.model,
                               modules=module,
                               x=xb[0], cpu=cpu)[0] for xb in aux_dl]
    embs = to_concat(embs)
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    if to_numpy: embs = embs.numpy() if cpu else embs.cpu().numpy()
    return embs

In [12]:
import wandb
from dvats.utils import *
wandb_api = wandb.Api()
#enc_artifact = wandb_api.artifact('tchub/learner-mvp:run-tchub-3tipekxw')
enc_artifact = wandb_api.artifact('mi-santamaria/test-project/dcae:v0')
enc_learner = enc_artifact.to_obj()
X = torch.rand(9, 1, 48)

wandb:   1 of 1 files downloaded.  


In [13]:
#slow
%%time
embs = get_enc_embs(X, enc_learner, cpu=True)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__

UsageError: Line magic function `%%time` not found.


In [14]:
%%time
embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=False)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__, embs.device

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
%%time
embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=True)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
beep(1)